# Librerías e importaciones

In [25]:
!pip install meteostat
!pip install geocoder
!pip install geopy

In [26]:
import pandas as pd
import json
import ast
import geocoder
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
from meteostat import Point, Hourly, Daily, Monthly
from fbprophet import Prophet
import datetime

# Exploración de datos

In [27]:
# Acceso a gdrive para datos
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/Saturdays_AI_DL"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
dfconsumption = pd.read_csv(path + "/holaluz/consumptions_small.csv", sep=';')
dfcups = pd.read_csv(path + "/holaluz/cups_info_small.csv", sep=';')
dfcalendario = pd.read_csv(path +"/festivities/calendario.csv", sep=';')

In [29]:
# Consumo (consumptions_small.csv)
dfconsumption.head()

,day,consumption,consumption_type,id
0,2019-12-01,"{1.139,2.04}","{TELEM,TELEM}",39653
1,2019-12-02,"{1.163,2.328}","{TELEM,TELEM}",39653
2,2019-12-03,"{0.873,2.578}","{TELEM,TELEM}",39653
3,2019-12-04,"{0.997,3.089}","{TELEM,TELEM}",39653
4,2019-12-05,"{1.818,3.694}","{TELEM,TELEM}",39653


In [30]:
# Consumidor (cups_info_small.csv)
dfcups.head()

,id,atr,power,zipcode,locality,ekon_start
0,39653,2.0TD,"{4600,4600}",28052,NaN,2021-09-01


In [31]:
# Festivos
dfcalendario.head()

,Día,Día_semana,laborable / festivo / domingo festivo,Tipo de Festivo,Festividad
0,01/01/2013,martes,festivo,Festivo nacional,Año Nuevo
1,02/01/2013,miércoles,laborable,NaN,NaN
2,03/01/2013,jueves,laborable,NaN,NaN
3,04/01/2013,viernes,laborable,NaN,NaN
4,05/01/2013,sábado,sábado,NaN,NaN


# Limpieza y preparación de datos

In [32]:

# Normalizamos los datos de consumo
dfconsumption.consumption = dfconsumption.consumption.apply(lambda x: ast.literal_eval(x.replace("{", "(").replace("}", ")")))
dfconsumption['total_consumption'] = dfconsumption.consumption.apply(lambda x: sum(x))

In [33]:
# Descartar lecturas de NO telemetría
dfconsumption = dfconsumption[dfconsumption.consumption_type.str.contains("TELEM")]

In [34]:
dfconsumption.head()

,day,consumption,consumption_type,id,total_consumption
0,2019-12-01,"(1.139, 2.04)","{TELEM,TELEM}",39653,3.179
1,2019-12-02,"(1.163, 2.328)","{TELEM,TELEM}",39653,3.491
2,2019-12-03,"(0.873, 2.578)","{TELEM,TELEM}",39653,3.451
3,2019-12-04,"(0.997, 3.089)","{TELEM,TELEM}",39653,4.086
4,2019-12-05,"(1.818, 3.694)","{TELEM,TELEM}",39653,5.512


In [35]:
# Hacemos join con los datos de las dos tablas, una contiene los datos de consumos y la otra los datos del consumidor
dfmerged = pd.merge(left=dfconsumption, right=dfcups, left_on='id', right_on='id')
dfmerged.head()

,day,consumption,consumption_type,id,total_consumption,atr,power,zipcode,locality,ekon_start
0,2019-12-01,"(1.139, 2.04)","{TELEM,TELEM}",39653,3.179,2.0TD,"{4600,4600}",28052,NaN,2021-09-01
1,2019-12-02,"(1.163, 2.328)","{TELEM,TELEM}",39653,3.491,2.0TD,"{4600,4600}",28052,NaN,2021-09-01
2,2019-12-03,"(0.873, 2.578)","{TELEM,TELEM}",39653,3.451,2.0TD,"{4600,4600}",28052,NaN,2021-09-01
3,2019-12-04,"(0.997, 3.089)","{TELEM,TELEM}",39653,4.086,2.0TD,"{4600,4600}",28052,NaN,2021-09-01
4,2019-12-05,"(1.818, 3.694)","{TELEM,TELEM}",39653,5.512,2.0TD,"{4600,4600}",28052,NaN,2021-09-01


In [36]:
# Necesitamos encontrar todos los diferentes zipcode del dataset y con sus fechas mínimas y máximas para la obtención de datos meteorológicos
df_zipcodes = dfmerged.groupby('zipcode').agg({'day': ['min', 'max']})
display(df_zipcodes.head())
df_zipcodes.info()

day            
                min         max
zipcode                        
28052    2019-12-01  2021-05-31

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 28052 to 28052
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   (day, min)  1 non-null      object
 1   (day, max)  1 non-null      object
dtypes: object(2)
memory usage: 24.0+ bytes


In [37]:
# Recorremos todos los zipcode y poblamos un nuevo dataset con zipcode, fechas y datos meteorológicos diarios
geolocator = Nominatim(user_agent='test')
df = pd.DataFrame(columns=['time', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun', 'zipcode'])
for index, x in df_zipcodes.iterrows():
  location = geolocator.geocode(str(x.name) + ", Spain")
  point = Point(location.latitude, location.longitude)
  data = Daily(point, pd.to_datetime(x.day.min()), pd.to_datetime(x.day.max()))
  data = data.fetch()
  # Obtenemos los datos de luz diaria con los datos mensuales
  dataMonth = Monthly(point, pd.to_datetime(x.day.min()), pd.to_datetime(x.day.max()))
  dataMonth = dataMonth.fetch()
  dataMonth = dataMonth.reset_index()
  # data.plot(y=['tavg', 'tmin', 'tmax'])
  data['zipcode'] = x.name
  data = data.reset_index()
  data['month'] = pd.to_datetime(data.time).dt.month
  data['year'] = pd.to_datetime(data.time).dt.year
  dataMonth['month'] = pd.to_datetime(dataMonth.time).dt.month
  dataMonth['year'] = pd.to_datetime(dataMonth.time).dt.year
  data = pd.merge(
    data, 
    dataMonth[['tsun', 'month', 'year']], 
    on=['month', 'year'],
    how='left')
  data['tsun'] = data['tsun_y']/30
  data = data.drop(['month', 'year', 'tsun_x', 'tsun_y'], axis = 1)
  df = df.append(data)
  print(pd.to_datetime(x.day.min()))
  print(pd.to_datetime(x.day.max()))
  print(location.address)
df_zipcodes.head()

# Los datos ya estarían listos para hacer join por el zipcode con los datos del consumer



2019-12-01 00:00:00
2021-05-31 00:00:00
Puente de Vallecas, Madrid, Área metropolitana de Madrid y Corredor del Henares, Comunidad de Madrid, 28052, España


day            
                min         max
zipcode                        
28052    2019-12-01  2021-05-31

In [38]:
# Tenemos que transformar el valor object a datetime
dfmerged['day'] =  pd.to_datetime(dfmerged['day'])
# Unimos las dos tablas a través del zipcode y la fecha
dfmerged = pd.merge(left=dfmerged, right=df, left_on=['zipcode', 'day'], right_on=['zipcode', 'time'], how='left')
dfmerged.head()

,day,consumption,consumption_type,id,total_consumption,atr,power,zipcode,locality,ekon_start,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2019-12-01,"(1.139, 2.04)","{TELEM,TELEM}",39653,3.179,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-01,7.4,6.2,8.7,26.8,NaN,71.2,7.8,NaN,1013.1,276.0
1,2019-12-02,"(1.163, 2.328)","{TELEM,TELEM}",39653,3.491,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-02,7.4,5.3,10.8,0.0,NaN,25.3,15.5,NaN,1016.1,276.0
2,2019-12-03,"(0.873, 2.578)","{TELEM,TELEM}",39653,3.451,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-03,5.3,2.8,9.0,2.0,NaN,11.2,10.7,NaN,1015.6,276.0
3,2019-12-04,"(0.997, 3.089)","{TELEM,TELEM}",39653,4.086,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-04,7.8,5.4,11.6,2.0,NaN,6.4,10.6,NaN,1011.5,276.0
4,2019-12-05,"(1.818, 3.694)","{TELEM,TELEM}",39653,5.512,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-05,9.3,7.9,12.0,0.0,NaN,10.9,12.0,NaN,1017.6,276.0


In [39]:
dfmerged.head(10000)

,day,consumption,consumption_type,id,total_consumption,atr,power,zipcode,locality,ekon_start,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2019-12-01,"(1.139, 2.04)","{TELEM,TELEM}",39653,3.179,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-01,7.4,6.2,8.7,26.8,NaN,71.2,7.8,NaN,1013.1,276.0
1,2019-12-02,"(1.163, 2.328)","{TELEM,TELEM}",39653,3.491,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-02,7.4,5.3,10.8,0.0,NaN,25.3,15.5,NaN,1016.1,276.0
2,2019-12-03,"(0.873, 2.578)","{TELEM,TELEM}",39653,3.451,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-03,5.3,2.8,9.0,2.0,NaN,11.2,10.7,NaN,1015.6,276.0
3,2019-12-04,"(0.997, 3.089)","{TELEM,TELEM}",39653,4.086,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-04,7.8,5.4,11.6,2.0,NaN,6.4,10.6,NaN,1011.5,276.0
4,2019-12-05,"(1.818, 3.694)","{TELEM,TELEM}",39653,5.512,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-05,9.3,7.9,12.0,0.0,NaN,10.9,12.0,NaN,1017.6,276.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,2021-05-27,"(1.571, 2.006)","{TELEM,TELEM}",39653,3.577,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2021-05-27,21.3,15.4,26.2,2.0,NaN,66.0,8.8,NaN,1015.2,604.0
544,2021-05-28,"(1.72, 3.096)","{TELEM,TELEM}",39653,4.816,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2021-05-28,20.7,15.3,26.2,0.0,NaN,100.4,6.9,NaN,1016.3,604.0
545,2021-05-29,"(0.713, 1.976)","{TELEM,TELEM}",39653,2.689,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2021-05-29,23.0,16.1,29.9,0.0,NaN,47.4,8.3,NaN,1018.0,604.0
546,2021-05-30,"(3.275, 3.238)","{TELEM,TELEM}",39653,6.513,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2021-05-30,24.3,17.7,30.6,0.0,NaN,22.5,10.7,NaN,1015.4,604.0


In [40]:
dfmerged.head()

,day,consumption,consumption_type,id,total_consumption,atr,power,zipcode,locality,ekon_start,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2019-12-01,"(1.139, 2.04)","{TELEM,TELEM}",39653,3.179,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-01,7.4,6.2,8.7,26.8,NaN,71.2,7.8,NaN,1013.1,276.0
1,2019-12-02,"(1.163, 2.328)","{TELEM,TELEM}",39653,3.491,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-02,7.4,5.3,10.8,0.0,NaN,25.3,15.5,NaN,1016.1,276.0
2,2019-12-03,"(0.873, 2.578)","{TELEM,TELEM}",39653,3.451,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-03,5.3,2.8,9.0,2.0,NaN,11.2,10.7,NaN,1015.6,276.0
3,2019-12-04,"(0.997, 3.089)","{TELEM,TELEM}",39653,4.086,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-04,7.8,5.4,11.6,2.0,NaN,6.4,10.6,NaN,1011.5,276.0
4,2019-12-05,"(1.818, 3.694)","{TELEM,TELEM}",39653,5.512,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-05,9.3,7.9,12.0,0.0,NaN,10.9,12.0,NaN,1017.6,276.0


# Prophet: Preparación de datos y Test Set



In [41]:
# Prophet necesita un dataset para los festivos y no laborales para valorar el impacto
# https://facebook.github.io/prophet/docs/seasonality,_holiday_effects,_and_regressors.html
# Sustituimos el formato de fecha actual y lo adaptamos a los otros datasets
# Prophet tambien nos pide una ventana de tiempo anterior y posterior a la fecha vacacional
dfcalendario["ds"] = pd.to_datetime(dfcalendario['Día'], format="%d/%m/%Y").dt.strftime("%Y-%m-%d")
dffestivities = pd.DataFrame({"ds": dfcalendario["ds"], 'lower_window': -3, 'upper_window' : 1, 'holiday': dfcalendario['laborable / festivo / domingo festivo']})
dffestivities.head()


,ds,lower_window,upper_window,holiday
0,2013-01-01,-3,1,festivo
1,2013-01-02,-3,1,laborable
2,2013-01-03,-3,1,laborable
3,2013-01-04,-3,1,laborable
4,2013-01-05,-3,1,sábado


In [42]:
# Convertimos a booleanos los festivos al nuevo dataset a partir del calendario de festivos
dffestivities['isholiday'] = (dffestivities['holiday'] == 'sábado') | (dffestivities['holiday'] == 'domingo') | (dffestivities['holiday'] == 'festivo')
# Cruzamos datos
dffestivities['day'] = dffestivities['ds'].astype('datetime64')
dfmerged = pd.merge(left=dfmerged, right=dffestivities[['isholiday','day']], left_on='day', right_on='day', how='left')
dfmerged.head(10)

,day,consumption,consumption_type,id,total_consumption,atr,power,zipcode,locality,ekon_start,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,isholiday
0,2019-12-01,"(1.139, 2.04)","{TELEM,TELEM}",39653,3.179,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-01,7.4,6.2,8.7,26.8,NaN,71.2,7.8,NaN,1013.1,276.0,False
1,2019-12-02,"(1.163, 2.328)","{TELEM,TELEM}",39653,3.491,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-02,7.4,5.3,10.8,0.0,NaN,25.3,15.5,NaN,1016.1,276.0,False
2,2019-12-03,"(0.873, 2.578)","{TELEM,TELEM}",39653,3.451,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-03,5.3,2.8,9.0,2.0,NaN,11.2,10.7,NaN,1015.6,276.0,False
3,2019-12-04,"(0.997, 3.089)","{TELEM,TELEM}",39653,4.086,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-04,7.8,5.4,11.6,2.0,NaN,6.4,10.6,NaN,1011.5,276.0,False
4,2019-12-05,"(1.818, 3.694)","{TELEM,TELEM}",39653,5.512,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-05,9.3,7.9,12.0,0.0,NaN,10.9,12.0,NaN,1017.6,276.0,False
5,2019-12-06,"(6.084, 3.62)","{TELEM,TELEM}",39653,9.704,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-06,9.1,6.7,12.5,0.0,NaN,0.1,6.0,NaN,1023.5,276.0,True
6,2019-12-07,"(0.881, 2.32)","{TELEM,TELEM}",39653,3.201,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-07,7.5,3.7,12.0,0.0,NaN,308.5,4.8,NaN,1025.8,276.0,False
7,2019-12-08,"(1.476, 2.016)","{TELEM,TELEM}",39653,3.492,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-08,5.6,4.1,7.0,2.7,NaN,356.9,3.0,NaN,1027.7,276.0,False
8,2019-12-09,"(1.603, 3.227)","{TELEM,TELEM}",39653,4.830,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-09,7.0,4.7,11.6,0.0,NaN,69.1,5.1,NaN,1030.1,276.0,True
9,2019-12-10,"(1.08, 4.116)","{TELEM,TELEM}",39653,5.196,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-10,5.2,3.9,8.0,0.0,NaN,338.5,5.0,NaN,1031.0,276.0,False


In [43]:
# Para festivities filtramos solo los que son festivo
dffestivities = dffestivities[dffestivities['isholiday'] == True]
dffestivities.head(10)

,ds,lower_window,upper_window,holiday,isholiday,day
0,2013-01-01,-3,1,festivo,True,2013-01-01
4,2013-01-05,-3,1,sábado,True,2013-01-05
5,2013-01-06,-3,1,domingo,True,2013-01-06
6,2013-01-07,-3,1,festivo,True,2013-01-07
11,2013-01-12,-3,1,sábado,True,2013-01-12
12,2013-01-13,-3,1,domingo,True,2013-01-13
18,2013-01-19,-3,1,sábado,True,2013-01-19
19,2013-01-20,-3,1,domingo,True,2013-01-20
25,2013-01-26,-3,1,sábado,True,2013-01-26
26,2013-01-27,-3,1,domingo,True,2013-01-27


In [83]:
# Prophet necesita la columna "y", variable a predecir y la columna "ds" como timestamp para el timeseries
# Por aquí
# Antes de entrenar el modelo escogeremos solo las columnas que son necesarias
dataset = dfmerged.loc[:, ('day', 'total_consumption', 'tavg', 'tmin', 'tmax', 'prcp', 'wdir', 'wspd', 'pres', 'tsun', 'isholiday')]
dataset['y'] = dfmerged['total_consumption']
dataset.rename(columns={"day": "ds"}, inplace=True)
dataset.ds.max()



Timestamp('2021-05-31 00:00:00')

In [95]:
# Existen algunos Nan que reemplazaremos por 0
dataset = dataset.fillna(0)

In [96]:
# Para el test vamos a hacer una predicción de un mes, eliminamos el último mes
test_days = 31
training_set = dataset.iloc[:-test_days, :]
test_set = dataset.iloc[-test_days:, :]
test_set.tail(1)


,ds,total_consumption,tavg,tmin,tmax,prcp,wdir,wspd,pres,tsun,isholiday,y
547,2021-05-31,5.202,23.5,18.2,30.9,6.3,199.8,12.6,1013.1,604.0,False,5.202


# Prophet: Modelo

In [97]:
# Facebook Prophet model. Holidays es un dataframe aparte que hemos construido
m = Prophet(growth="linear", 
            yearly_seasonality=True,
            weekly_seasonality=True, 
            daily_seasonality=False, 
            holidays=dffestivities, 
            seasonality_mode = "multiplicative", 
            seasonality_prior_scale = 10,
            holidays_prior_scale= 0.05)
# Agregamos regresores al modelo
# Controlamos también isholiday como regressor del dataframe de dataset y todos los datos climatológicos
m.add_regressor('isholiday')
# Temperatura media
m.add_regressor('tavg')
# Temperatura mínima
m.add_regressor('tmin')
# Temperatura máxima
m.add_regressor('tmax')
# Precipitaciones
m.add_regressor('prcp')
# Dirección del viento
m.add_regressor('wdir')
# Velocidad del viento
m.add_regressor('wspd')
# Minutos de sol
m.add_regressor('tsun')
# Entrenamos el modelo
m.fit(training_set)

# Forecasting

In [98]:
# Creamos el Future Dataframe, será el dataframe con los valores relativos al futuro. 
# La frecuencia será "D", un día
future = m.make_future_dataframe (periods = len(test_set), freq = "D")
# Vemos que los datos corresponden al último día del dataframe de test
print('dataset', dataset.tail(1))
print('training_set', training_set.tail(1))
print('test_set', test_set.tail(1))
print('future', future.tail(1))

dataset             ds  total_consumption  tavg  tmin  ...    pres   tsun  isholiday      y
547 2021-05-31              5.202  23.5  18.2  ...  1013.1  604.0      False  5.202

[1 rows x 12 columns]
training_set             ds  total_consumption  tavg  tmin  ...    pres   tsun  isholiday      y
516 2021-04-30              2.433  11.6   7.7  ...  1009.6  388.0      False  2.433

[1 rows x 12 columns]
test_set             ds  total_consumption  tavg  tmin  ...    pres   tsun  isholiday      y
547 2021-05-31              5.202  23.5  18.2  ...  1013.1  604.0      False  5.202

[1 rows x 12 columns]
future             ds
547 2021-05-31


In [99]:
# Juntamos los regressors del dataset al future
future = pd.concat([future, dataset.iloc[:,2:]], axis = 1)
future.tail()

,ds,tavg,tmin,tmax,prcp,wdir,wspd,pres,tsun,isholiday,y
543,2021-05-27,21.3,15.4,26.2,2.0,66.0,8.8,1015.2,604.0,False,3.577
544,2021-05-28,20.7,15.3,26.2,0.0,100.4,6.9,1016.3,604.0,False,4.816
545,2021-05-29,23.0,16.1,29.9,0.0,47.4,8.3,1018.0,604.0,False,2.689
546,2021-05-30,24.3,17.7,30.6,0.0,22.5,10.7,1015.4,604.0,False,6.513
547,2021-05-31,23.5,18.2,30.9,6.3,199.8,12.6,1013.1,604.0,False,5.202


In [100]:
# Forecast. Hacemos nuestra predicción
forecast = m.predict(future)
forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,domingo,domingo_lower,domingo_upper,extra_regressors_multiplicative,extra_regressors_multiplicative_lower,extra_regressors_multiplicative_upper,festivo,festivo_lower,festivo_upper,holidays,holidays_lower,holidays_upper,isholiday,isholiday_lower,isholiday_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,prcp,prcp_lower,prcp_upper,sábado,sábado_lower,sábado_upper,tavg,tavg_lower,tavg_upper,tmax,tmax_lower,tmax_upper,tmin,tmin_lower,tmin_upper,tsun,tsun_lower,tsun_upper,wdir,wdir_lower,wdir_upper,weekly,weekly_lower,weekly_upper,wspd,wspd_lower,wspd_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2019-12-01,5.935112,2.005463,7.689777,5.935112,5.935112,0.0,0.0,0.0,-0.077154,-0.077154,-0.077154,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,-0.174766,-0.174766,-0.174766,-0.048263,-0.048263,-0.048263,0.0,0.0,0.0,0.001428,0.001428,0.001428,0.005249,0.005249,0.005249,-0.019810,-0.019810,-0.019810,-0.02714,-0.02714,-0.02714,0.016106,0.016106,0.016106,-0.029866,-0.029866,-0.029866,-0.004724,-0.004724,-0.004724,-0.067746,-0.067746,-0.067746,0.0,0.0,0.0,4.897855
1,2019-12-02,5.938342,2.168528,8.177687,5.938342,5.938342,0.0,0.0,0.0,-0.014151,-0.014151,-0.014151,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,-0.107714,-0.107714,-0.107714,0.002904,0.002904,0.002904,0.0,0.0,0.0,0.001428,0.001428,0.001428,0.004147,0.004147,0.004147,-0.025206,-0.025206,-0.025206,-0.02714,-0.02714,-0.02714,0.022875,0.022875,0.022875,-0.035934,-0.035934,-0.035934,0.006841,0.006841,0.006841,-0.057629,-0.057629,-0.057629,0.0,0.0,0.0,5.298700
2,2019-12-03,5.941573,2.312175,8.176865,5.941573,5.941573,0.0,0.0,0.0,-0.036668,-0.036668,-0.036668,0.015886,0.015886,0.015886,0.015886,0.015886,0.015886,0.0,0.0,0.0,-0.126561,-0.126561,-0.126561,-0.000914,-0.000914,-0.000914,0.0,0.0,0.0,0.001903,0.001903,0.001903,0.005091,0.005091,0.005091,-0.040193,-0.040193,-0.040193,-0.02714,-0.02714,-0.02714,0.024954,0.024954,0.024954,-0.057468,-0.057468,-0.057468,-0.000368,-0.000368,-0.000368,-0.048310,-0.048310,-0.048310,0.0,0.0,0.0,5.189603
3,2019-12-04,5.944803,2.006018,8.378609,5.944803,5.944803,0.0,0.0,0.0,-0.022452,-0.022452,-0.022452,-0.007762,-0.007762,-0.007762,-0.007762,-0.007762,-0.007762,0.0,0.0,0.0,-0.099296,-0.099296,-0.099296,-0.000914,-0.000914,-0.000914,0.0,0.0,0.0,0.001338,0.001338,0.001338,0.003727,0.003727,0.003727,-0.024606,-0.024606,-0.024606,-0.02714,-0.02714,-0.02714,0.025661,0.025661,0.025661,-0.029380,-0.029380,-0.029380,-0.000519,-0.000519,-0.000519,-0.039701,-0.039701,-0.039701,0.0,0.0,0.0,5.354509
4,2019-12-05,5.948034,2.667997,8.766629,5.948034,5.948034,0.0,0.0,0.0,-0.002756,-0.002756,-0.002756,-0.007574,-0.007574,-0.007574,-0.007574,-0.007574,-0.007574,0.0,0.0,0.0,-0.045655,-0.045655,-0.045655,0.002904,0.002904,0.002904,0.0,0.0,0.0,0.000999,0.000999,0.000999,0.003517,0.003517,0.003517,-0.009619,-0.009619,-0.009619,-0.02714,-0.02714,-0.02714,0.024998,0.024998,0.024998,-0.003642,-0.003642,-0.003642,0.001584,0.001584,0.001584,-0.031683,-0.031683,-0.031683,0.0,0.0,0.0,5.676479
